In [1]:
import xlrd
import csv
import pandas as pd
from tqdm import tqdm

# This function spits a csv file in your curr working directory
def xls2csv(filepath, sheet):
    #wb = xlrd.open_workbook('4omics-commonPatients/inputfiles_process/'+filename+'.xlsx')
    wb = xlrd.open_workbook(filepath)
    sh = wb.sheet_by_name(sheet)
    your_csv_file = open(filepath.split('.')[0]+'.csv', 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)
    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))
    your_csv_file.close()
    return None

In [2]:
# # DONOT, I repeat DONOT run this section if csv files are already available
# #Convert xlsx omics file to csv for easy parsing
# xls2csv('validation_files\Input\CNV_CommonPatientsAndGenes_TCGA_DATA.xlsx', 'Sheet1')
# xls2csv('validation_files\Input\meth_Commongenes-and-CommonTCGAPatients_DATA.xlsx', 'Sheet1')
# xls2csv('validation_files\Input\Rna_CommonPatientsAndGenes_TCGA_DATA.xlsx', 'Sheet1')
# xls2csv('validation_files\Input\Mutation_CommonPatientsAndGenes_TCGA_DATA.xlsx', 'Sheet1')
# # xls2csv('bioinfo16', 'Sheet1')

In [3]:
# Import list of NCA selected genes
# Change the path as required
cnv_genes = pd.read_csv('4omics-commonPatients\inputfiles_process\cnv_genes.csv', header=None).values.tolist()
meth_genes = pd.read_csv('4omics-commonPatients\inputfiles_process\meth_genes.csv', header=None).values.tolist()
mrna_genes = pd.read_csv('4omics-commonPatients\inputfiles_process\mrna_genes.csv', header=None).values.tolist()
mut_genes = pd.read_csv('4omics-commonPatients\inputfiles_process\mut_genes.csv', header=None).values.tolist()
len(cnv_genes[0]), len(meth_genes[0]), len(mrna_genes[0]), len(mut_genes[0])

(26, 9, 15, 26)

In [4]:
# Import the DR validation file
# Change the path as required
bioinfo16=pd.read_csv('csv_files/bioinfo16.csv')[['bcr_patient_barcode', 'drug_name', 'measure_of_response']]
bioinfo16['bcr_patient_barcode'] = bioinfo16['bcr_patient_barcode'].astype(str)+'_01'
bioinfo16 = bioinfo16[bioinfo16['drug_name'].isin(['Vinblastine', 'Gemcitabine', 'Tamoxifen', 'Docetaxel'])] 

for i in bioinfo16.index:
    if bioinfo16.loc[i]['measure_of_response'] in ['Stable Disease', 'Clinical Progressive Disease']:
        bioinfo16.loc[i]['measure_of_response']=1
    elif bioinfo16.loc[i]['measure_of_response'] in ['Complete Response', 'Partial Response']:
        bioinfo16.loc[i]['measure_of_response']=0

In [5]:
# INPORT VALIDATION FILES
# Convert xlsx omics file to csv for easy parsing
cnv = pd.read_csv('validation_files/Input/CNV_CommonPatientsAndGenes_TCGA_DATA.csv').set_index('Unnamed: 0').loc[cnv_genes[0],:]
meth = pd.read_csv('validation_files/Input/meth_Commongenes-and-CommonTCGAPatients_DATA.csv').set_index('Unnamed: 0').T.loc[meth_genes[0],:]
mrna = pd.read_csv('validation_files/Input/Rna_CommonPatientsAndGenes_TCGA_DATA.csv').set_index('Unnamed: 0').loc[mrna_genes[0],:]
mut = pd.read_csv('validation_files/Input/Mutation_CommonPatientsAndGenes_TCGA_DATA.csv').set_index('Unnamed: 0').loc[mut_genes[0],:]

In [6]:
# To extract common patients for validation
cnv_val=cnv[list(set(list(cnv)) & set(list(bioinfo16['bcr_patient_barcode'])))]
mrna_val=mrna[list(set(list(mrna)) & set(list(bioinfo16['bcr_patient_barcode'])))]
mut_val=mut[list(set(list(mut)) & set(list(bioinfo16['bcr_patient_barcode'])))]
meth_val=meth[list(set(list(cnv)) & set(list(bioinfo16['bcr_patient_barcode'])))]

# Save vaildation files to xlsx
cnv_val.to_excel("validation_files_out_2/cnv_val.xlsx"), mrna_val.to_excel("validation_files_out_2/mrna_val.xlsx") , mut_val.to_excel("validation_files_out_2/mut_val.xlsx") , meth_val.to_excel("validation_files_out_2/meth_val.xlsx") 
print(cnv_val.shape, meth_val.shape, mrna_val.shape, mut_val.shape)

(26, 32) (9, 32) (15, 32) (26, 32)


In [7]:
# Import MATLAB results of drug responses for validation cohort
# model_y_val.csv is the validation output for 66 common patients
# DR.txt is a refernce to extract drug names
y_val=pd.read_csv('model_y_val2.csv', header=None).T
y_val.columns=list(pd.read_csv('DR.txt', sep=','))
y_val.index=list(mrna_val)

#Extract data of common drugs
DR_val=y_val[list(set(list(y_val)) & set(bioinfo16['drug_name'].unique()))]
print('Common Drugs:', list(DR_val))

Common Drugs: ['Docetaxel', 'Gemcitabine', 'Vinblastine', 'Tamoxifen']


In [8]:
# Binarize drug responses to resistant and senstive 
# 0=Resistant
# 1= Sensitive

thres = pd.read_csv('thres.csv', header=None)
thres.index=list(pd.read_csv('DR.txt', sep=','))

for index in tqdm(DR_val.index): 
    for drug in list(DR_val): 
        if DR_val[drug][index]>=DR_val.mean(axis=0)[drug]:
        #if DR_val[drug][index]>thres.loc[drug].values[0]:
            DR_val[drug][index]=1
        elif DR_val[drug][index]<DR_val.mean(axis=0)[drug]:
        #elif DR_val[drug][index]<=thres.loc[drug].values[0]:
            DR_val[drug][index]=0

100%|██████████| 32/32 [00:00<00:00, 191.62it/s]


In [20]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

# Extract total patients for each drug
ForDocetaxel = bioinfo16[bioinfo16['drug_name']=='Docetaxel']
ForVinblastine = bioinfo16[bioinfo16['drug_name']=='Vinblastine']
ForTamoxifen = bioinfo16[bioinfo16['drug_name']=='Tamoxifen']
ForGemcitabine = bioinfo16[bioinfo16['drug_name']=='Gemcitabine']

print('Number of patients available for each drug:', len(intersection(ForTamoxifen['bcr_patient_barcode'].values, DR_val.index)),
        len(intersection(ForVinblastine['bcr_patient_barcode'].values, DR_val.index)),
        len(intersection(ForDocetaxel['bcr_patient_barcode'].values, DR_val.index)),
        len(intersection(ForGemcitabine['bcr_patient_barcode'].values, DR_val.index)))

# Predication for each drug
Tamoxifen_predicted = DR_val[DR_val.index.isin(intersection(ForTamoxifen['bcr_patient_barcode'].values, DR_val.index))]['Tamoxifen']
Vinblastine_predicted = DR_val[DR_val.index.isin(intersection(ForVinblastine['bcr_patient_barcode'].values, DR_val.index))]['Vinblastine']
Docetaxel_predicted = DR_val[DR_val.index.isin(intersection(ForDocetaxel['bcr_patient_barcode'].values, DR_val.index))]['Docetaxel']
Gemcitabine_predicted = DR_val[DR_val.index.isin(intersection(ForGemcitabine['bcr_patient_barcode'].values, DR_val.index))]['Gemcitabine']

# True value for each drug
Tamoxifen_true = ForTamoxifen[ForTamoxifen['bcr_patient_barcode'].isin(intersection(ForTamoxifen['bcr_patient_barcode'].values, DR_val.index))]
Vinblastine_true = ForVinblastine[ForVinblastine['bcr_patient_barcode'].isin(intersection(ForVinblastine['bcr_patient_barcode'].values, DR_val.index))]
Docetaxel_true = ForDocetaxel[ForDocetaxel['bcr_patient_barcode'].isin(intersection(ForDocetaxel['bcr_patient_barcode'].values, DR_val.index))]
Gemcitabine_true = ForGemcitabine[ForGemcitabine['bcr_patient_barcode'].isin(intersection(ForGemcitabine['bcr_patient_barcode'].values, DR_val.index))]

Number of patients available for each drug: 11 1 24 1


In [22]:
# Calculate accuracy
def calAcc(predicted, true):
    score = 0
    for patient in predicted.index:
        if true[true['bcr_patient_barcode']==patient]['measure_of_response'].values[0]==predicted[patient]:
            score+=1
    accuracy = score/len(predicted.index)
    return(accuracy)

print('Tamoxifin:', calAcc(Tamoxifen_predicted, Tamoxifen_true), '(11 patients)')
print('Vinblastine:',calAcc(Vinblastine_predicted, Vinblastine_true), '(1 patients)')
print('Docetaxel:',calAcc(Docetaxel_predicted, Docetaxel_true), '(24 patients)')
print('Gemcitabine:',calAcc(Gemcitabine_predicted, Gemcitabine_true), '(1 patients)')

Tamoxifin: 0.5 (11 patients)
Vinblastine: 1.0 (1 patients)
Docetaxel: 0.7916666666666666 (24 patients)
Gemcitabine: 1.0 (1 patients)
